In [ ]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [2]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.248.58 49974 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '8448', 'VSCODE_CLI_REQUIRE_TOKEN': 'e2fff225-9583-46c0-8e69-bfcf24b4c518', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-fabdb6a30b49f79a7aba0f2ad9df9b399473380f/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT

In [ ]:
"""
Test the vina docking and extracting score.
"""

import sys
import os


from datetime import datetime

from REVIVAL.zs.vina import dock_lib
from REVIVAL.util import checkNgen_folder

if __name__ == "__main__":

    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    # log outputs
    f = open(
        os.path.join(
            checkNgen_folder("logs/zs/vina/dock"),
            f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}.out",
        ),
        "w",
    )
    sys.stdout = f

    for chai_dir in [ 
        "zs/chai/mut_structure/PfTrpB-5chloro-plp", 
        "zs/chai/mut_structure/PfTrpB-4cyano-plp",
        "zs/chai/mut_structure/PfTrpB-5chloro-plp",
        "zs/chai/mut_structure/PfTrpB-6chloro-plp",
        "zs/chai/mut_structure/PfTrpB-7bromo-plp",
        "zs/chai/mut_structure/PfTrpB-7indo-plp",
        "zs/chai/mut_structure/PfTrpB-7methyl-plp",
        "zs/chai/mut_structure/PfTrpB-56chloro-plp",
        "zs/chai/mut_structure/PfTrpB-5cyano-plp",
    ]:
        dock_lib(chai_dir, "inactivated-cofactor")


    # def dock_lib(
    #     chai_dir: str,
    #     cofactor_type: str,
    #     vina_dir: str = "vina",
    #     pH: float = 7.4,
    #     method='vina',
    #     size_x=15.0,
    #         size_y=15.0, 
    #         size_z=15.0,
    #         num_modes=9, # Dunno check vina docks using the defaut
    #         exhaustiveness=32 
    # ):

    f.close()

In [5]:

def extract_lowest_energy(file_path: str):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    for i, line in enumerate(lines):
        if "mode |   affinity | dist from best mode" in line:
            table_start = i + 3  # Skip to the actual table
            break
    
    energies = []
    for line in lines[table_start:]:
        if line.strip() == "":  # End of the table
            break
        parts = line.split()
        try:
            energies.append(float(parts[1]))  # Affinity is in the second column
        except ValueError:
            pass  # Skip invalid lines
    
    return min(energies)


In [6]:
extract_lowest_energy(file_path="/disk2/fli/REVIVAL2/zs/vina/PfTrpB-4cyano_inactivated-cofactor/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0-4cyano_log.txt")

-11.81

In [19]:
from scipy.stats import spearmanr
import pandas as pd
import numpy as np

In [16]:
df_56chloro = pd.read_csv("/disk2/fli/REVIVAL2/zs/vina/score/PfTrpB-56chloro_inactivated-cofactor.csv")
df_56chloro

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,cofactor,cofactor-smiles,intermediate-smiles,vina_0,vina_1,vina_2,vina_3,vina_4,vina,vina_rank
0,AAV,0.011308,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.580,-9.670,-11.730,-11.430,-11.570,-11.1960,204.0
1,ADL,0.013558,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-7.158,-10.180,-10.860,-8.463,-9.652,-9.2626,118.0
2,AGC,0.012568,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.230,-10.890,-10.230,-11.050,-11.140,-10.9080,190.0
3,AGL,0.026299,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.070,-10.640,-10.250,-11.310,-11.580,-10.9700,193.0
4,AMN,0.018304,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.570,-11.450,-11.000,-8.851,-11.750,-10.9242,191.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,YSQ,0.005915,I165Y:I183S:Y301Q,3,Y,S,Q,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.940,-6.069,-11.450,-10.090,-9.608,-9.8314,132.0
237,YTG,0.040378,I165Y:I183T:Y301G,3,Y,T,G,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-12.160,-7.507,-9.582,-11.220,-11.670,-10.4278,168.0
238,YTM,0.046069,I165Y:I183T:Y301M,3,Y,T,M,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-8.678,-3.413,-7.183,-6.400,-6.149,-6.3646,54.0
239,YVA,0.199846,I165Y:I183V:Y301A,3,Y,V,A,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,-11.830,-8.475,-10.870,-9.716,-8.738,-9.9258,135.0


In [18]:
df_56chloro[df_56chloro["vina"].isna()]

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,cofactor,cofactor-smiles,intermediate-smiles,vina_0,vina_1,vina_2,vina_3,vina_4,vina,vina_rank
82,IIE,0.166763,Y301E,1,I,I,E,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,IIF,1.010853,Y301F,1,I,I,F,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,IIG,0.138453,Y301G,1,I,I,G,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,IIH,1.046647,Y301H,1,I,I,H,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,III,0.012910,Y301I,1,I,I,I,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,IIK,0.025817,Y301K,1,I,I,K,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,IIL,0.020196,Y301L,1,I,I,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,IIM,0.068034,Y301M,1,I,I,M,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,IIQ,0.045930,Y301Q,1,I,I,Q,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,IIT,0.031893,Y301T,1,I,I,T,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,56chloro,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC(Cl)=C...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
x = df_56chloro["fitness"].values
y = df_56chloro["vina"].values
mask = ~np.isnan(x) & ~np.isnan(y)  # Mask to exclude NaNs in both arrays
spearmanr(x[mask], y[mask])

SignificanceResult(statistic=0.11744643497796997, pvalue=0.07546696248269448)

In [6]:
df = pd.read_csv("/disk2/fli/REVIVAL2/zs/vina/score/PfTrpB-4cyano_inactivated-cofactor.csv")
df

,AAs,fitness,var,n_mut,AA1,AA2,AA3,seq,enzyme,substrate,...,cofactor,cofactor-smiles,intermediate-smiles,vina_0,vina_1,vina_2,vina_3,vina_4,vina,vina_rank
0,AAV,0.179022,I165A:I183A:Y301V,3,A,A,V,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-11.810,-12.310,-11.68,-11.880,-11.720,-11.8800,237.0
1,ADL,-0.164331,I165A:I183D:Y301L,3,A,D,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-8.275,-10.750,-11.00,-7.789,-10.460,-9.6548,126.0
2,AGC,0.039124,I165A:I183G:Y301C,3,A,G,C,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-11.960,-11.070,-10.90,-11.560,-11.410,-11.3800,219.0
3,AGL,0.119577,I165A:I183G:Y301L,3,A,G,L,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-11.600,-11.310,-11.07,-11.840,-11.190,-11.4020,221.0
4,AMN,0.271701,I165A:I183M:Y301N,3,A,M,N,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-12.220,-12.400,-11.19,-10.640,-11.760,-11.6420,232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,YSQ,-0.000670,I165Y:I183S:Y301Q,3,Y,S,Q,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-12.760,-9.825,-11.39,-11.590,-9.312,-10.9754,198.0
237,YTG,-0.036533,I165Y:I183T:Y301G,3,Y,T,G,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-12.140,-9.842,-10.49,-9.497,-9.903,-10.3744,161.0
238,YTM,-0.041777,I165Y:I183T:Y301M,3,Y,T,M,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-9.545,-8.088,-10.51,-5.376,-7.923,-8.2884,81.0
239,YVA,0.171030,I165Y:I183V:Y301A,3,Y,V,A,MWFGEFGGQYVPETLVGPLKELEKAYKRFKDDEEFNRQLNYYLKTW...,PfTrpB,4cyano,...,PLP-dependent aminoacrylate.Na+,[O-]C1=C(/C=[N+]([H])/C(C([O-])=O)=C)C(CP([O-]...,CC1=C([O-])C(\C=[NH+]\[C@@H](CC2=CNC3=CC=CC(C#...,-12.730,-10.100,-11.63,-10.250,-9.953,-10.9326,193.0


In [8]:
spearmanr(df["fitness"], df["vina_rank"])

SignificanceResult(statistic=0.049859401255100994, pvalue=0.441014538964538)